# Into To Deploying App To AWS

Since there is so much to know about AWS, this will mostly focus on a specific use case:
_How to get your app up & running on a server on the internet  so you can test it with friends & family._

We will use the **AWS Free Tier**. We will be creating a new server on AWS, deploying our code, & configuring the server to make it available on the internet.

You will need to sign up for the AWS free tier [here](https://aws.amazon.com/free/?all-free-tier.sort-by=item.additionalFields.SortRank&all-free-tier.sort-order=asc&awsf.Free%20Tier%20Types=*all&awsf.Free%20Tier%20Categories=*all).

# Add Key Pair to AWS

In thsi section, we will add an SSH key pair to AWS. This will be done so that when we create our server, we can connect to it using SSH authentication. This is the same method of authentication used to connect to GitHub.

In prior vieo we created an SSH key paid on our local machine. We will now be using the public key to add to AWS insetad of GitHub.

## Output the content of public key pair

If you don't have your SSH yet, follow the instructions [here](https://docs.github.com/en/authentication/connecting-to-github-with-ssh/checking-for-existing-ssh-keys).

1. Open **git bash** or terminal window - NOT on the vagrant server


2. Run:  `cat ~/.ssh/id_rsa.pub`

    Be sure to copy everything - including the email.
    
    
3. After logging into AWS console, go into the **Services** option and **EC2**. This si where the key pairs can be added for use on server instances.


4.  Under **Network & Security** go to **Key Pairs**. If you don't already see it, click on the **Actions** drop down and select **Import key pair**.

<img src='files/IMGs/AWS/AWS01.png'>


5. Give it a name you'll remember (and if using more than one machine, ensure you can tell the difference). Then paste the output of your RSA data in the box.


6. Click **Import Key** to successfully import the key.

<img src='files/IMGs/AWS/AWS02.png'>

# Create EC2 Server Instance

This EC2 server instance will run our project. This _instance_ is a virtual machine that you can spin up and & connect to in order to deploy your application.

1. Open AWS console & select EC2 - if you are not already in the **EC2 Dashboard** be sure to click there. This is where you will find all of your EC2 server instances.

<img src='files/IMGs/AWS/AWS03.png'>


2. Click on **Launch instance** to create a new instance. If you are using the free tier 12 month period, you should not be charged as long as you use the options for the free tier.

<img src='files/IMGs/AWS/AWS04.png'>

3. Give your instance a name. In the **Aplication and Os Images (Amazon Machine Image)** section, click the **Browse more AMIs** and then locate the one mentioned [here](https://gist.github.com/LondonAppDev/d195610d79c9cca1769b6b5fece09067) (`Ubuntu Server 18.04`).

<img src='files/IMGs/AWS/AWS05.png'>

    Then click **Select** button.
    
<img src='files/IMGs/AWS/AWS06.png'>

4. Be sure to choose the **micro** instance if it's not chosen already. (2nd one down.)

<img src='files/IMGs/AWS/AWS07.png'>


5. Choose your key pair for login using the one you just added.

<img src='files/IMGs/AWS/AWS08.png'>

6. In the **Network settings** area, be sure to check **_Allow HTTP traffic from the internet

<img src='files/IMGs/AWS/AWS09.png'>


7. Click the **Launch instance** button.

You should now see a screen similar to:

<img src='files/IMGs/AWS/AWS10.png'>

At this point, you can set up billing alerts if oyu wish to know when you are coming close to your free tier limits.

# [Add Deployment Script & Configs To Project](https://www.udemy.com/course/django-python/learn/lecture/7371366#questions)

## Add Files To Project

1. Download the **deploy.zip** file [here](https://drive.google.com/file/d/1axRytyrpIAUN5K_LOyMnIlr5ZwdXBglV/view). Thsi file contains a ZIp of all the scripts & configuration files needed to deploy the project. (We'll add the contents of this to our project.)

2. Unzip the downloaded ZIP & copy the _deploy directory_ into your profiles REST API project:

<img src='files/IMGs/AWS/AWS11.png'>

## Review Of Files

In Atom, you should now see something like this:

<img src='files/IMGs/AWS/AWS12.png'>

### setup.sh

This script that will be used to setup th server when it's first created. Please note that an updated version should be found [here](https://gist.github.com/LondonAppDev/7090d8097553ab82209adb1809304d0e).

Need to update the project URL for your GitHub project. This is because the script will clone the contents of our project to the server when we run it. You can find the correct URL on the GitHub site under the **Code** drop down:

<img src='files/IMGs/AWS/AWS13.png'>

This URL needds to be in the **PROJECT_GIT_URL** variable.

The **PROJECT_BASE_PATH** is where the project will be kept on the AWS server.

In the **apt-get install** we will install:
- git:  clone the project
- nginx:  the web server that will serve the static files & act as a proxy to our uWSGI service that will run in **supervisor**

Other installations:
- uwsgi:  Python daemon for running python code as a web server

The `manage.py collectstatic --noinput` will collect all of the static files for all of the apps in our project into one directory. When you use the Django management server, it does this automatically for you. But since we're not going to be using the Django DEV server on PROD, we need to create a location with all of the static files we can use to serve the CSS & JavaScript for the Django admin & REST framework browseable API.

Configuring the **supervisor** - an app on linux that allows you to manage processes. Thsi is what will manage our Python process or our uWSGI server.

### update.sh

This script will be used to update the code on the server when we make a change. Once the server is setup, **setup.sh** will only work once. You'll then need to use the **update.sh** script to pull updates or changes from git onto the server.

### Update settings.py

In the project folder, you'll need to update the **setings.py** file for it to run better on the server. Here are the settings changes to be made:

1. Disable DEBUG mode, as it is best practice to nbever run in DEBUG mode when it is publicly accessible. If you leave it as `True`, it opens you up to vulnerabilities. Running in DEBUG mode means that if there are any errors or exceptions in the API it will return a full stack trace on the screen. This can reveal secret information like the Django secret file or other things making your server vulnerable.

    Since we want to run DEBUG on vagrant DEV server but not on AWS, replace `DEBUG = True` with:

In [ ]:
DEBUG = bool(int(os.environ.get('DEBUG', 1)))  # the 1 is for if "DEBUG" doesn't exist

This will pull in the value of the environment variable called DEBUG. This environment variable is set in the **supervisor_profiles_api.conf** file. When **DEBUG=0**, this sets the debug environment variable to 0 when we run our application.

2. Need to add a static route to the bottom of your configuration.

In [ ]:
STATIC_ROOT = 'static/'

The **STATIC_ROOT** is the location where Django will store all of the static files when we run our collect static command in the **setup.sh** file.

There is 1 more command that needs to be run to ensure the setup scripts are executable.

when you send a file to a server, the file needs to have the permissions to be executable if you want to run it as an executable script. This can be done by:

1. Opening the terminal in **git bash** in your project


2. Run `chmod +x deploy/*.sh` - this runs the **chmod** command to set executable any file that ends in **.sh** in our deployed directory.

# Deploy To Server

Next we'll SSH to our server & run our deploy script.

## SSH Connect To Server

In AWS, go to **Services > EC2 > Instances** to locate your server. Click on your server, and information about it should show below, like this:

<img src='files/IMGs/AWS/AWS14.png'>

Locate your **Public IPv4 DNS** section and copy the full URL.

In your **git bash** terminal, and run:  `ssh ubuntu@PUBLIC_URL_FOR_SERVER` (without the http:// or final `/`)

**ubuntu** is the user we will connect to & authentication should automatically happen via our public key. It should then look something like this:

<img src='files/IMGs/AWS/AWS15.png'>

Say `yes` and provide the passphrase, if you had one. You should now be logged in:

<img src='files/IMGs/AWS/AWS16.png'>

## Download & Run The Script To Setup Server

Go to your GitHub project & refresh the page. Go to your **deploy** folder & get the URL for your **raw** data file that contains your **setup.sh** script.

In your SSH connection to your AWS server, run a command similar to:  `curl -sL https://raw.githubusercontent.com/USERNAME/REPO/main/deploy/setup.sh | sudo bash -`

This will run the CURL command to download the file, then pass it into **sudo bash** ... or to retrieve content from a URL. (It's basically a HTTP client in Linux.)

The `-s` is for running in **silent mode** - meaning it won't update us with all of the steps when it's downloading the file.

The `L` is for following redirects.

The `|` is to output one command into another.

The `sudo` is used to run commands as administrator on linux.

`bash` is what we will use to run our script.

The `-` is used to signal the end of the options provided for `bash` so it knows anything should be run and not confiugred on `bash`.

## Confirm No Access Via Browser

Once script completed running, head over to the Amazon Web Services (AWS) console & copy the hostname again. Put this into the browser - if you see 404, it's because the hostname has not been added to the settings.py file yet.

<img src='files/IMGs/AWS/AWS17.png'>

The above image is not a 404, but it is what I had when I followed the instructions for this course.

# Update Allowed Hosts & Deploy Changes

Here is where we will update the allowed host setting in Django configuration in order to resolve "bad request" error.

In your project folder, locate the `settings.py` file. At the top of the file, locate the `ALLOWED_HOSTS` variable. This is a setting that allows us to enable access via specific domain names. Itis a security feature to ensure if somebody finds a random IP address for our server, they will not be able to access the application unless they use a valid hostname.

The hostname you need to add is the hostname for your server in AWS EC2 configuration. Specifically, the public DNS option. Paste that as a string into the list.

You will also need to add the local host, so it still works on your vagrant server. Therefore, you will need to add:  `12.0.0.1`

It should look similar to the following:
<img src='files/IMGs/AWS/AWS18.png'>

Once these have been added, save the file.

## How To update Changes After Pushed To GitHub